In [26]:
import nascar_api
from pathlib import Path

In [27]:
parent_dir = Path.cwd().parent
inator_dir = parent_dir.joinpath("collect_inator")
inator_dir.mkdir(exist_ok=True)

In [ ]:
confirm = input("THIS WILL OVERWRITE GENERATED ""_base"" FILES AND __init__ FILE.\nThis will NOT touch existing ""_logic"" files.\nContinue (Y/N)? ")

if confirm.lower() == "y":
   init_string = ""
   import_string = """#######################################################################
## THIS FILE IS AUTOMATICALLY GENERATED. DO NOT MAKE CHANGES HERE.
#######################################################################

from toolbox.nascar_api import Feeds, get_api_data
from sqlalchemy import Engine 
import pandas as pd
"""
   script_string = "\n\n"

   for fi, f in enumerate(nascar_api.Feeds):
      #print(fi, f.name, f.value)
      p = f.value.params
      param_string = (", " + ", ".join([f"{x}: int" for x in p])) if p != [] else ""
      dict_string = ", ".join([f"\"{x}\": {x}" for x in p])

      init_string += f"from .collect_inator_base import get_{f.name.lower()}\n"
      
      import_string += f"from collect_inator import {f.name.lower()}_logic\n"

      script_string += f"""def get_{f.name.lower()}(url_header: str{param_string}, sql_engine: Engine | None = None, load_to_sql: bool = True) -> tuple[bool, pd.DataFrame]:
   api_result = get_api_data(Feeds.{f.name}, {{{dict_string}}}, url_header=url_header)

   if api_result["result"]:
      df = {f.name.lower()}_logic.json_to_df(api_result["json"])
      
      if load_to_sql:
         if sql_engine == None: return (False, pd.DataFrame())
         existing_df = pd.read_sql_table(table_name="{f.name.lower()}", con=sql_engine, schema="nascar")
         df = pd.concat([df, existing_df])
         df.drop_duplicates().to_sql("{f.name.lower()}", con=sql_engine, if_exists="append", index=False)

      return (True, df)
   else:
      print(f"API call failed for Feeds.{f.name}. URL response code = {{api_result["result_code"]}}")
      return (False, pd.DataFrame())\n\n"""
      
      logic_dir = inator_dir.joinpath(f"{f.name.lower()}_logic").with_suffix(".py")
      if not logic_dir.exists():
         logic_dir.touch()
         logic_dir.write_text("""import pandas as pd

def json_to_df(json: object) -> pd.DataFrame:
   df = pd.DataFrame(json) # type: ignore
   return df""")
         
   
   script_dir = inator_dir.joinpath("collect_inator_base").with_suffix(".py")

   script_dir.touch()
   script_dir.write_text(import_string + script_string)
         
   init_dir = inator_dir.joinpath("__init__").with_suffix(".py")
   init_dir.write_text(init_string)